In [1]:
data = {
  "metrics": [
    {
      "name": "average_gmv_local",
      "label": "Average GMV Local",
      "description": "The average checkout price excluding SH fees (base and/or event fees). This only accounts rentals with 'payment_status' = 'success' and 'reservation_status' = 'valid'",
      "type": "DERIVED",
      "typeParams": {
        "expr": "gmv_local / rentals_sold"
      },
      "measures": [
        {
          "name": "spothero_fee_revenue_local_valid_measure",
          "agg": "SUM",
          "expr": "CASE WHEN is_valid = True THEN spothero_fee_rollup_local ELSE 0 END"
        },
        {
          "name": "count_of_rentals_sold_measure",
          "agg": "SUM",
          "expr": "CASE WHEN is_valid THEN 1 ELSE 0 END"
        },
        {
          "name": "gov_local_valid_measure",
          "agg": "SUM",
          "expr": "CASE WHEN is_valid = True THEN rental_checkout_price_local ELSE 0 END"
        }
      ],
      "config": {
        "meta": {
          "group": "Core",
          "metric_type": "Operational Financial Metrics",
          "metric_category": "Financial Metrics"
        }
      },
      "entities": [
        {
          "name": "rental",
          "expr": "rental_id"
        },
        {
          "name": "forex",
          "expr": "forex_id"
        },
        {
          "name": "facility",
          "expr": "facility_configuration_id"
        },
        {
          "name": "event",
          "expr": "event_id"
        },
        {
          "name": "renter",
          "expr": "renter_id"
        },
        {
          "name": "partner",
          "expr": "partner_id"
        },
        {
          "name": "canonical_facility",
          "expr": "canonical_facility_id"
        },
        {
          "name": "operator",
          "expr": "operator_id"
        },
        {
          "name": "geography",
          "expr": "geo_id"
        },
        {
          "name": "destination",
          "expr": "destination_id"
        }
      ]
    }
  ],
  "dimensions": [
    {
      "name": "renter__phone_number",
      "label": "Phone Number",
      "description": "Thie field denotes the default phone number of the renter.",
      "type": "CATEGORICAL",
      "dimensionTags": {
        "data_group": None,
        "name": "renter__phone_number",
        "primary_tag": "Renter Details",
        "secondary_tag": "Details"
      }
    },
    {
      "name": "renter__last_name",
      "label": "Last Name",
      "description": "This field denotes the last name of the renter.",
      "type": "CATEGORICAL",
      "dimensionTags": {
        "data_group": None,
        "name": "renter__last_name",
        "primary_tag": "Renter Details",
        "secondary_tag": "Details"
      }
    }
  ]
}
import json

import lkml

import looker_models as lm
from app import app, db
from app.models import metrics_schema, dimensions_schema

graphql_metrics = data['metrics']
graphql_dimensions = data['dimensions']
#retool_view

sqlite:////Users/doug/Documents/pythonProjects/abacus/abacus.db


In [2]:
with app.app_context(), db.session.no_autoflush:
    metrics = metrics_schema.load(graphql_metrics)
    dimensions = dimensions_schema.load(graphql_dimensions)

In [ ]:
tablename = "rental_performance"
schema = "dbt_looker"
sql_trigger_value = "SELECT FLOOR(((TIMESTAMP_DIFF(CURRENT_TIMESTAMP(),'1970-01-01 00:00:00',SECOND)) - 60*60*10)/(60*60*24))"
derived_table = {}
derived_table['sql'] = f"select * from {schema}.{tablename}"
derived_table['sql_trigger_value'] = sql_trigger_value
derived_table = {"derived_table": derived_table}

view = {"name": tablename}

view.update(derived_table)
view.update({"measures": [lm.to_looker(metric) for metric in metrics]})
view.update({"dimensions": [lm.to_looker(dimension) for dimension in dimensions]})

view = {"view": view}
print(type(view))
lookml = lkml.dump(view)

# with open('explore.lkml', 'w') as f:
#     for line in lookml:
#         f.write(f"{line}")
